# Default Import

In [1]:
from ris import pysqldb
from IPython.display import Markdown, clear_output
from sqlalchemy import create_engine
import ris
import getpass
import datetime 
import pandas as pd
import numpy as np
import os
import requests

timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')
ts = datetime.datetime.now().strftime('%Y-%m-%d')

Markdown('<strong>Notebook run on: {} | by {} | Using ris library version: {} | File Location: {}'.format(
    timestamp, getpass.getuser(), ris.__version__, os.getcwd()
))


<strong>Notebook run on: 2021-05-21 17:35 | by soge | Using ris library version: 2.2.1 | File Location: E:\RIS\Staff Folders\Samuel\Requests\SH\Speed Limits\051821_Targee_St

# Db connections

In [2]:
try: 
    db.params['user']
except:
    db = pysqldb.DbConnect(type='PG', server='dotdevrhpgsql01', database='CRASHDATA')


Additional database connection details required:
User name (crashdata):soge
Password (crashdata)········


In [3]:
#sip = pysqldb.DbConnect(server='dotpgsql01', database='sip', type='PG', user = db.params['user'], password = db.params['password'])
#forms = pysqldb.DbConnect(server='dot55sql01', database='forms', type='MS', user = 'arcgis', password = 'arcgis')

#BACK UP SPEED LIMITS FILE
#CREATE A TABLE WITH SPEED LIMITS DATA AND TODAY'S DATE
#i.e Archive Copy

In [4]:
db.query("""
CREATE TABLE archive_speed_limit_20d_52121 AS

SELECT * FROM speed_limit_20d;

GRANT ALL ON archive_speed_limit_20d_52121 TO PUBLIC;
""")

- Query run 2021-05-21 17:36:25.294000
 Query time: Query run in 179000 microseconds 
 * Returned 0 rows *


Streets: 

Milford Avenue from Ocean Terrace to Clove Road

In [5]:
speed_limits = pd.read_excel('vzv_speed_limits_update.xlsx')

In [6]:
speed_limits.loc[speed_limits.street == 'TARGEE STREET']

,segmentid,street,postvz_sl,postvz_sg,prevz_sl,prevz_sg,version
0,15334,TARGEE STREET,30,YES,30,YES,20d
1,15344,TARGEE STREET,30,YES,30,YES,20d
2,15346,TARGEE STREET,30,YES,30,YES,20d


In [7]:
targee_st = tuple(map(int,list(speed_limits.loc[speed_limits.street == 'TARGEE STREET'].segmentid)))
other_st = tuple(map(int,list(speed_limits.loc[speed_limits.street != 'TARGEE STREET'].segmentid)))

### Inserting into Speed limits table if necessary

insert = db.dfquery("""
INSERT INTO speed_limit_19d(segmentid, street, postvz_sl, postvz_sg, prevz_sl, prevz_sg, version, geom)
SELECT segmentid::int , street, 0 postvz_sl, 'NO'::varchar postvz_sg, 0 prevz_sl, 'NO'::varchar _prevz_sg, '19d' "version", geom
FROM LION
WHERE segmentid ::int IN  ('0312733', '0312802', '0312769')
""")

In [20]:
x = db.dfquery("""
SELECT segmentid, street, postvz_sl, postvz_sg, prevz_sl, prevz_sg, 
       version, geom
FROM public.speed_limit_20d
WHERE segmentid ::int IN  {} 
""".format(other_st))

In [27]:
x.loc[x.street == 'MILFORD DRIVE']

,segmentid,street,postvz_sl,postvz_sg,prevz_sl,prevz_sg,version,geom
65,12806,MILFORD DRIVE,30,NO,30,NO,20d,0105000020D70800000100000001020000000200000000...
66,12944,MILFORD DRIVE,30,NO,30,NO,20d,0105000020D70800000100000001020000000200000080...
67,13095,MILFORD DRIVE,30,NO,30,NO,20d,0105000020D70800000100000001020000000200000080...
68,13100,MILFORD DRIVE,30,NO,30,NO,20d,0105000020D70800000100000001020000000200000080...
69,13107,MILFORD DRIVE,30,NO,30,NO,20d,0105000020D70800000100000001020000000200000000...
70,13112,MILFORD DRIVE,30,NO,30,NO,20d,0105000020D70800000100000001020000000200000080...
71,13116,MILFORD DRIVE,30,NO,30,NO,20d,0105000020D70800000100000001020000000200000000...


In [16]:
#segs = tuple(map(int,list(x.segmentid)))
segs = tuple(map(int,segments))

In [11]:
y = db.query("""UPDATE public.speed_limit_20d
                SET postvz_sl = 30
                WHERE segmentid in {}
                """.format(targee_st))

- Query run 2021-05-21 17:50:01.686000
 Query time: Query run in 24000 microseconds 
 * Returned 0 rows *
